In [1]:
import os
import ee
import geemap
import socket
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio
from shapely.geometry import box 
from shapely.geometry import mapping
import re # regular expressions
import folium
from folium import GeoJson
from folium.plugins import MarkerCluster
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.features import geometry_mask
import json
import warnings

def setup_directories():
    # check if we are on the server or local
    nodename = socket.gethostname()
    if nodename == "oMac.local": # local laptop
        root = os.path.expanduser("~/OneDrive - The University of Chicago/guatamala_ag/data")
    else:
        raise Exception("Unknown environment, Please specify the root directory")

    dirs = {
        'root': root,
        'raw': os.path.join(root, "raw"),
        'processed': os.path.join(root, "processed"),
        'fig': os.path.join(root, "../figures")
    }

    for path in dirs.values():
        os.makedirs(path, exist_ok=True)

    return dirs

dir = setup_directories()

### Clean Coordinate data and then convert to geopandas, and calculate area

In [2]:
def add_decimal_if_missing(coord):
    if isinstance(coord, str) and coord.replace('-', '').isdigit():
        if coord.startswith('-'):
            return f"-{coord[1:3]}.{coord[3:]}"
        else:
            return f"{coord[:2]}.{coord[2:]}"
    return coord

def is_valid_guatemala_coordinate(lat, lon):
    # Approximate bounding box for Guatemala
    return 13.1 <= lat <= 18.2 and -93 <= lon <= -88.0

def fix_known_coordinate_issues(df):
    """
    Fix known coordinate issues in the dataframe using approximate floating-point comparisons.
    """
    known_fixes = {
        ('longitude_4', 90.366662): -90.366662,
        ('latitude_1', 5.267439): 15.267439,
        ('longitude_1', -16.352620): -90.1635620,
    }
    
    for (col, incorrect_value), correct_value in known_fixes.items():
        # Use numpy's isclose for approximate floating-point comparison
        mask = np.isclose(df[col], incorrect_value, rtol=1e-5, atol=1e-8)
        if mask.any():
            df.loc[mask, col] = correct_value
            print(f"Fixed {mask.sum()} occurrences of approximately {incorrect_value} to {correct_value} in {col}")
    
    return df

def split_coordinates(coord_str):
    # Existing manual fixes
    manual_fixes = {
        "16,3870407, -89,7345351": "16.3870407, -89.7345351",
        "14.177150.3,-90.3989608": "14.1771503, -90.3989608",
        "16,3869101, -89,7344694": "16.3869101, -89.7344694",
        "14.141996-90-147208": "14.141996, -90.147208",
        "16,3871767, -89,7348127": "16.3871767, -89.7348127",
        "16,3869863, -89,7349780": "16.3869863, -89.7349780"
    }
    
    if coord_str in manual_fixes:
        coord_str = manual_fixes[coord_str]
    
    # Remove any quotation marks and leading/trailing whitespace
    cleaned = coord_str.strip().strip('"')
    
    # Try to match various patterns
    patterns = [
        r'^([-]?\d+\.?\d*)[,\s]+([-]?\d+\.?\d*)$',  # Comma or space separated
        r'^([-]?\d+\.?\d*)\.([-]?\d+\.?\d*)$',      # Period separated
        r'^(\d+\.?\d*)(-\d+\.?\d*)$'                # No separator with negative longitude
    ]
    
    for pattern in patterns:
        # if we get passed the first pattern, remove all whitespace
        cleaned = re.sub(r'\s', '', cleaned)
        match = re.match(pattern, cleaned)
        if match:
            lat, lon = match.group(1), match.group(2)
            lat = add_decimal_if_missing(lat)
            lon = add_decimal_if_missing(lon)
            return pd.Series({'latitude': lat, 'longitude': lon})
    
    # If we couldn't split it, return empty strings
    print(f"Could not split coordinates: {coord_str}")
    return pd.Series({'latitude': '', 'longitude': ''})

df = pd.read_excel(os.path.join(dir['raw'], "Datos de Impacto Productores 2023.xlsx"), 
    sheet_name= 0, skiprows=4)
vars_to_keep = ["id_phone", "id_coordinates_1", "id_coordinates_2", 
                "id_coordinates_3", "id_coordinates_4"]
df = df[vars_to_keep]
# drop rows with missing id_coordinates_1
df = df.dropna(subset=["id_coordinates_1"])


# Process coordinates
for i in range(1, 5):
    col_name = f'id_coordinates_{i}'
    new_cols = df[col_name].apply(split_coordinates)
    df[f'latitude_{i}'] = pd.to_numeric(new_cols['latitude'], errors='coerce')
    df[f'longitude_{i}'] = pd.to_numeric(new_cols['longitude'], errors='coerce')
    
    # Check if coordinates are within Guatemala's range
    df[f'valid_coordinate_{i}'] = df.apply(
        lambda row: is_valid_guatemala_coordinate(row[f'latitude_{i}'], row[f'longitude_{i}']), 
        axis=1
    )

# Fix known coordinate issues
df = fix_known_coordinate_issues(df)

# Recheck validity after fixes
for i in range(1, 5):
    df[f'valid_coordinate_{i}'] = df.apply(
        lambda row: is_valid_guatemala_coordinate(row[f'latitude_{i}'], row[f'longitude_{i}']), 
        axis=1
    )

# Print summary of remaining invalid coordinates
for i in range(1, 5):
    invalid_coords = df[~df[f'valid_coordinate_{i}']]
    if not invalid_coords.empty:
        print(f"\nRemaining invalid coordinates for id_coordinates_{i}:")
        print(invalid_coords[[f'latitude_{i}', f'longitude_{i}']])

# Check if all coordinates are valid
all_valid = df.apply(lambda row: all(row[f'valid_coordinate_{i}'] for i in range(1, 5)), axis=1)
print(f"\nTotal rows with all valid coordinates: {all_valid.sum()} out of {len(df)}")

# Save the processed data
df.to_csv(os.path.join(dir['processed'], "coordinates_processed.csv"), index=False)
print("\nData processing complete. Results saved to 'coordinates_processed.csv'.")

Fixed 1 occurrences of approximately 90.366662 to -90.366662 in longitude_4
Fixed 1 occurrences of approximately 5.267439 to 15.267439 in latitude_1
Fixed 1 occurrences of approximately -16.35262 to -90.163562 in longitude_1

Total rows with all valid coordinates: 125 out of 125

Data processing complete. Results saved to 'coordinates_processed.csv'.


/Users/ohouck/envs/guatamala_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell LH147 is marked as a date but the serial value 6729160 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [3]:
# create lat and lon min and max columns
df['lat_min'] = df[['latitude_1', 'latitude_2', 'latitude_3', 'latitude_4']].min(axis=1)
df['lat_max'] = df[['latitude_1', 'latitude_2', 'latitude_3', 'latitude_4']].max(axis=1)
df['lon_min'] = df[['longitude_1', 'longitude_2', 'longitude_3', 'longitude_4']].min(axis=1)
df['lon_max'] = df[['longitude_1', 'longitude_2', 'longitude_3', 'longitude_4']].max(axis=1)

# Function to create a polygon from min/max coordinates
# jury is out on which is better
def create_polygon(row):
    return box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max'])

# Alternative Function to create a polygon from coordinates
# def create_polygon(row):
#     coords = [
#         (float(row['longitude_1']), float(row['latitude_1'])),
#         (float(row['longitude_2']), float(row['latitude_2'])),
#         (float(row['longitude_3']), float(row['latitude_3'])),
#         (float(row['longitude_4']), float(row['latitude_4'])),
#         (float(row['longitude_1']), float(row['latitude_1']))  # Close the polygon
#     ]
#     return Polygon(coords)


# Create the geometry column
df['geometry'] = df.apply(create_polygon, axis=1)

crs = "EPSG:5459" # crs for guatemala
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=crs)

# Function to calculate area in square meters
def calculate_area(geometry, lat):
    # Define a local projection centered on the polygon
    local_azimuthal_projection = f"+proj=aeqd +lat_0={lat}\
        +lon_0={geometry.centroid.x} +x_0=0 +y_0=0"
    
    # Create a GeoSeries with the input geometry and set its CRS
    geoseries = gpd.GeoSeries([geometry], crs="EPSG:4326")
    
    # Project the GeoSeries to the local azimuthal equidistant projection
    projected_geoseries = geoseries.to_crs(local_azimuthal_projection)
    
    # Get the projected geometry and calculate its area
    projected_geometry = projected_geoseries.iloc[0]

    area = projected_geometry.area

    # check if area is nan
    if pd.isna(area):
        print(f"Area is nan for {geometry}")
        # print the lat and lon
        print(f"Lat: {lat}, Lon: {geometry.centroid.x}")
        return

    return area 

# create an id column
gdf['id'] = range(len(gdf))

# Calculate area for each polygon
gdf['area_sqm'] = gdf.apply(lambda row: calculate_area(row['geometry'], 
                            row['geometry'].centroid.y), axis=1)

# count number of rows with area over 1 million sqm
print(f"Number of rows with area over 500k sqm: {len(gdf[gdf['area_sqm'] > 500_000])}")

# drop rows with area over 0.5 million sqm (5k by 5k meters)
gdf = gdf[gdf['area_sqm'] < 500_000]

# Display info about the GeoDataFrame
print(f"\nGeoDataFrame shape: {gdf.shape}")
print(f"GeoDataFrame CRS: {gdf.crs}")

# print summary statistics for area and round to 2 decimal places
print(gdf['area_sqm'].describe().round(2))

# save the geodataframe
gdf.to_file(os.path.join(dir['processed'], "cleaned_parcels.geojson"), driver='GeoJSON')

Number of rows with area over 500k sqm: 8

GeoDataFrame shape: (117, 24)
GeoDataFrame CRS: EPSG:5459
count       117.00
mean      12407.17
std       31590.46
min           0.00
25%        1884.04
50%        4174.59
75%       10579.19
max      243667.68
Name: area_sqm, dtype: float64


### Function to Pull Sentinel 2 images


In [ ]:
def get_sentinel2_imagery_multi(geometries, ids, start_date, end_date, output_dir, 
                                bands_to_save, max_cloud_cover=10):
    ee.Initialize()

    
    for geom, id in zip(geometries, ids):
        try:
            ee_geometry = ee.Geometry.Polygon(list(geom.exterior.coords))
            
            s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                             .filterBounds(ee_geometry)
                             .filterDate(start_date, end_date)
                             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)))
            
            if s2_collection.size().getInfo() == 0:
                print(f"No images found for geometry {id}. Skipping.")
                continue

            # normalized difference vegetation index
            def addNDVI(image):
                ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
                return image.addBands(ndvi)

            # green chlorophyl vegetation index (not using but used in SCYM)
            def addGCVI(image):
                gcvi = image.normalizedDifference(['B8', 'B3']).rename('GCVI')  # B8: NIR, B3: Green
                return image.addBands(gcvi)

            # Sort by NDVI and select the image with the highest NDVI
            s2_collection = s2_collection.sort('NDVI', False)
            selected_image = ee.Image(s2_collection.first())
            
            clipped = selected_image.clip(ee_geometry)
            output_file = os.path.join(output_dir, f"sentinel_image_{id}.tif")

            geemap.ee_export_image(clipped, filename=output_file, scale=10, region=ee_geometry)

            # Save band information
            band_info = {i+1: name for i, name in enumerate(bands_to_save)}
            with open(os.path.join(output_dir, f"sentinel_image_{id}_bands.json"), 'w') as f:
                json.dump(band_info, f)
            
            print(f"Sentinel image for geometry {id} saved to {output_file}")
            
        except Exception as e:
            print(f"Error processing geometry {id}: {str(e)}")

    print("All individual Sentinel images have been saved.")

# Usage
geometries = gdf.geometry.tolist()
ids = gdf.id.tolist()

# take 10 percent sample for testing
geometries = geometries[::10]
ids = ids[::10]

output_dir = os.path.join(dir['processed'], "sentinel_images")
os.makedirs(output_dir, exist_ok=True)

# currently saving all bands but might not be necessary
bands_to_save = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 
                    'B9', 'B10', 'B11', 'B12', "NDVI", "id"]

get_sentinel2_imagery_multi(geometries, ids, '2023-01-01', '2023-06-30', 
                            output_dir, bands_to_save)

In [8]:
# read in a single raster to test
raster = rasterio.open(os.path.join(output_dir, "sentinel_image_10.tif"))

# combine with the bands info
with open(os.path.join(output_dir, "sentinel_image_10_bands.json"), 'r') as f:
    bands = json.load(f)

# print band info
print(bands)

# print raster info
print(raster.meta)

{'1': 'B1', '2': 'B2', '3': 'B3', '4': 'B4', '5': 'B5', '6': 'B6', '7': 'B7', '8': 'B8', '9': 'B8A', '10': 'B9', '11': 'B10', '12': 'B11', '13': 'B12', '14': 'NDVI', '15': 'id'}
{'driver': 'GTiff', 'dtype': 'uint32', 'nodata': None, 'width': 4, 'height': 3, 'count': 23, 'crs': CRS.from_epsg(32615), 'transform': Affine(10.0, 0.0, 798610.0,
       0.0, -10.0, 1632110.0)}


## Combine pulled Sentinel images into a single raster


In [25]:
import os
import rasterio
import numpy as np
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from geopandas import GeoDataFrame

# Suppress specific warnings
warnings.filterwarnings("ignore", message="TIFFReadDirectory")

def get_band_info(file_path):
    json_path = file_path.rsplit('.', 1)[0] + '_bands.json'
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            return json.load(f)
    return None

def combine_tif_files(input_dir, output_file, bands_to_save):
    # Get all .tif files in the input directory
    tif_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.tif')]
    
    # Initialize lists to store the bounds and open dataset objects
    bounds_list = []
    src_files_to_mosaic = []
    band_info_list = []
    
    # Open all files and collect their bounds and band info
    for file in tif_files:
        src = rasterio.open(file)
        bounds_list.append(src.bounds)
        src_files_to_mosaic.append(src)
        band_info = get_band_info(file)
        if band_info:
            band_info_list.append(band_info)
        else:
            print(f"Warning: No band info found for {file}")
    
    # Determine available bands from band info
    if band_info_list:
        available_bands = list(band_info_list[0].values())
        print(f"Available bands: {available_bands}")
    else:
        available_bands = src_files_to_mosaic[0].descriptions
        print(f"Warning: Using default band names. Available bands: {available_bands}")
    
    # Filter bands_to_save to only include available bands
    bands_to_save = [band for band in bands_to_save if band in available_bands]
    if not bands_to_save:
        raise ValueError("None of the specified bands are available in the source files.")
    
    print(f"Bands to be saved: {bands_to_save}")
    
    # Read the first file to get the metadata
    meta = src_files_to_mosaic[0].meta.copy()
    
    # Update the metadata for the output file
    meta.update({
        'count': len(bands_to_save),
        'nodata': 0,
        'dtype': 'float32'
    })
    
    # Calculate the overall bounds
    bounds = rasterio.coords.BoundingBox(
        left=min(bound.left for bound in bounds_list),
        bottom=min(bound.bottom for bound in bounds_list),
        right=max(bound.right for bound in bounds_list),
        top=max(bound.top for bound in bounds_list)
    )
    
    # Create a mask for the farm parcels
    mask_shape = box(*bounds)
    geo = GeoDataFrame({'geometry': [mask_shape]}, crs=src_files_to_mosaic[0].crs)
    
    try:
        # Perform the mosaic operation
        mosaic, out_trans = merge(src_files_to_mosaic)
        
        # Update the metadata with the new transform and shape
        meta.update({
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans
        })

        # Create a mask for areas outside the farm parcels
        mask = geometry_mask([mapping(geo.geometry.iloc[0])], out_shape=mosaic.shape[1:], transform=out_trans, invert=True)
        
        # Apply the mask to the mosaic
        mosaic = np.ma.masked_array(mosaic, np.broadcast_to(mask, mosaic.shape))
        
        # Write the output file
        with rasterio.open(output_file, "w", **meta) as dest:
            for i, band_name in enumerate(bands_to_save):
                if band_info_list:
                    band_index = list(band_info_list[0].values()).index(band_name)
                else:
                    band_index = available_bands.index(band_name)
                dest.write(mosaic[band_index], i+1)
                dest.set_band_description(i+1, band_name)
        
        print(f"Combined raster saved to {output_file}")
    
    finally:
        # Close all opened raster files
        for src in src_files_to_mosaic:
            src.close()


    
# Usage
input_dir = os.path.join(dir['processed'], "sentinel_images")
output_file = os.path.join(dir['processed'], "merged_sentinel_image.tif")
bands_to_save = ['B2', 'B3', 'B4', 'B8', 'NDVI']  # Adjust as needed

combine_tif_files(input_dir, output_file, bands_to_save)

Available bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'NDVI', 'id']
Bands to be saved: ['B2', 'B3', 'B4', 'B8', 'NDVI']


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as 

Combined raster saved to /Users/ohouck/OneDrive - The University of Chicago/guatamala_ag/data/processed/merged_sentinel_image.tif


### Create Interactive Map

In [ ]:
import os
import rasterio
import numpy as np
import folium
from folium import raster_layers
import geopandas as gpd
from rasterio.warp import transform_bounds
from pyproj import Transformer
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_interactive_map(raster_file, parcel_file, output_file, max_zoom=10):
    try:
        logging.info("Starting to create interactive map")
        
        # Read the raster file
        with rasterio.open(raster_file) as src:
            logging.info(f"Opened raster file: {raster_file}")
            
            # Get raster metadata without reading all the data
            bounds = src.bounds
            transform = src.transform
            crs = src.crs
            
            # Transform bounds to lat/lon (EPSG:4326)
            transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
            minx, miny = transformer.transform(bounds.left, bounds.bottom)
            maxx, maxy = transformer.transform(bounds.right, bounds.top)
        
        logging.info("Transformed raster bounds to lat/lon")

        # Read the parcel shapefile
        parcels = gpd.read_file(parcel_file)
        logging.info(f"Read parcel file: {parcel_file}")

        # Create a map centered on the raster data
        m = folium.Map(location=[(miny + maxy) / 2, (minx + maxx) / 2], zoom_start=min(10, max_zoom))

        # Add the raster layer as a TileLayer for memory efficiency
        url = f"http://localhost:8080/raster/{os.path.basename(raster_file)}" + "/{z}/{x}/{y}.png"
        raster_layer = folium.raster_layers.TileLayer(
            tiles=url,
            attr="Raster Data",
            name="Raster Layer",
            overlay=True,
            opacity=0.7
        )
        raster_layer.add_to(m)

        logging.info("Added raster layer to map")

        # Add parcel boundaries
        folium.GeoJson(
            parcels,
            name="Farm Parcels",
            style_function=lambda feature: {
                'fillColor': 'blue',
                'color': 'black',
                'weight': 2,
                'fillOpacity': 0.1,
            }
        ).add_to(m)

        logging.info("Added parcel boundaries to map")

        # Add a layer control
        folium.LayerControl().add_to(m)

        # Save the map
        m.save(output_file)
        logging.info(f"Interactive map saved to {output_file}")

    except Exception as e:
        logging.error(f"An error occurred: {str(e)}", exc_info=True)
        raise

raster_file = os.path.join(dir['processed'], "merged_sentinel_image.tif")
parcel_file= os.path.join(dir['processed'], "cleaned_parcels.geojson") 
output_file = os.path.join(dir['fig'], "interactive_map.html")

create_interactive_map(raster_file, parcel_file, output_file)

: 